In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn import metrics

from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from scipy.optimize import brentq

In [2]:
# PARAMETERS
# define filepath to read data
dir_path = 'data/Surgical_data/'
# Randomized Seed
seed = 123
# Testing Set Proportion
test_proportion = 0.25

In [3]:
# read data
data = pd.read_csv((dir_path+'Surgical-deepnet.csv'), sep =  ",",na_values="?")
#Turn the outcome into an independent vector
y_data = data.pop('complication')

In [4]:
y_data.value_counts()

0    10945
1     3690
Name: complication, dtype: int64

In [5]:
#Split between training and testing set
X_train_inc_cal, X_test, y_train_inc_cal, y_test = train_test_split(data, y_data, test_size = test_proportion, random_state = seed)

In [6]:
#Split between training and calibration set
X_train, X_cal, y_train, y_cal = train_test_split(X_train_inc_cal, y_train_inc_cal, test_size = 0.5, random_state = seed)

In [7]:
#Train the classifier
# clf = RandomForestClassifier() 

# param_grid = {'n_estimators': [100, 200, 250, 500, 750, 1000],
#                  'max_depth': [2, 3, 4, 5, 6, 7, 10, 15]
#              }

In [8]:
# 10-Fold Cross validation
# grid_clf = GridSearchCV(clf, param_grid, cv=10)
# grid_clf.fit(X_train, y_train)

In [9]:
# score = grid_clf.score(X_test, y_test) 
# print(score)

# y_hat_test = grid_clf.predict(X_test)
# cm = metrics.confusion_matrix(y_test, y_hat_test)
# print(cm)

# #Seaborn Matrix
# plt.figure(figsize=(9,9))
# sns.heatmap(cm, annot=True, fmt=".0f", linewidths=.5, square = True, cmap = 'Blues_r');
# plt.ylabel('Actual label');
# plt.xlabel('Predicted label');
# all_sample_title = 'Accuracy Score: {0}'.format(score)
# plt.title(all_sample_title, size = 15); 

In [10]:
#Output probabilities
# y_proba_hat_train = grid_clf.predict_proba(X_train)
# y_proba_hat_cal = grid_clf.predict_proba(X_cal)
# y_proba_hat_test = grid_clf.predict_proba(X_test)

In [11]:
# new_col_names = ['proba_0','proba_1']

# X_train['y'] = pd.Series(y_train)
# X_cal['y'] = pd.Series(y_cal)
# X_test['y'] = pd.Series(y_test)

# X_train[new_col_names] = pd.DataFrame(y_proba_hat_train).set_index(X_train.index)
# X_cal[new_col_names] = pd.DataFrame(y_proba_hat_cal).set_index(X_cal.index)
# X_test[new_col_names] = pd.DataFrame(y_proba_hat_test).set_index(X_test.index)

# X_train.to_csv(dir_path+'Surgical_data_train_processed_seed_'+str(seed)+'.csv')
# X_cal.to_csv(dir_path+'Surgical_data_cal_processed_seed_'+str(seed)+'.csv')
# X_test.to_csv(dir_path+'Surgical_data_test_processed_seed_'+str(seed)+'.csv')

### Conformal risk control setup

In [12]:
X_train = pd.read_csv(dir_path + 'Surgical_data_train_processed_seed_' + str(seed) + '.csv', index_col=0)
X_cal = pd.read_csv(dir_path + 'Surgical_data_cal_processed_seed_' + str(seed) + '.csv', index_col=0)
X_test = pd.read_csv(dir_path + 'Surgical_data_test_processed_seed_' + str(seed) + '.csv', index_col=0)

In [13]:
# Problem setup
n = len(X_cal) # number of calibration points
alpha = 0.1 # 1-alpha is the desired false negative rate

In [14]:
def risk_metrics(y_pred, y_true):
    FP = np.logical_and(y_true != y_pred, y_true == 0).sum()  
    FN = np.logical_and(y_true != y_pred, y_true == 1).sum()  
    TP = np.logical_and(y_true == y_pred, y_true == 1).sum()  
    TN = np.logical_and(y_true == y_pred, y_true == 0).sum()  
    
    FNR = 1. * FN/(TP + FN)  
    FPR = 1. * FP/(FP + TN)  
    
    return pd.DataFrame([FNR, FPR]).T.rename(columns = {0:'FNR', 1:'FPR'})

In [17]:
# calibration and validation sets 
cal_sgmd = X_cal['proba_1']
test_sgmd = X_test['proba_1']
cal_gt = X_cal['y']
test_gt = X_test['y']

In [42]:
cal_sgmd.shape

(5488,)

In [27]:
# Run the conformal risk control procedure
def lamhat_threshold(risk_type, lam): 
    risk_metric = risk_metrics(cal_sgmd >= lam, cal_gt)[risk_type]
    return risk_metric - ((n+1)/n*alpha - 1/(n+1)) ###is 1/(n+1) an error ? not that it will matter much

lamhat = brentq(lambda lam: lamhat_threshold('FNR', lam), 0, 1)
y_test_pred = test_sgmd >= lamhat

In [28]:
# Calculate empirical FNR
print(f"The empirical FNR is: {risk_metrics(y_test_pred, test_gt)['FNR'][0]} and the threshold value is: {lamhat}")

The empirical FNR is: 0.12665198237885464 and the threshold value is: 0.19448080741201598


### Inverse CP

In [22]:
lam = 0.2

In [29]:
def alphahat_threshold(risk_type, alpha): 
    risk_metric = risk_metrics(cal_sgmd >= lam, cal_gt)[risk_type]
    return risk_metric - ((n+1)/n*alpha - 1/(n+1)) 

alphahat_FNR = brentq(lambda alpha: alphahat_threshold('FNR', alpha), 0, 1)
alphahat_FPR = brentq(lambda alpha: alphahat_threshold('FPR', alpha), 0, 1)
y_test_pred = test_sgmd >= lam

In [30]:
# Calculate empirical FNR
print(f"Empirical FNR: {risk_metrics(y_test_pred, test_gt)['FNR'][0]}")
print(f"Empirical FPR: {risk_metrics(y_test_pred, test_gt)['FPR'][0]}")
print(f"FNR alpha estimate: {alphahat_FNR}")
print(f"FNR alpha estimate: {alphahat_FPR}")

Empirical FNR: 0.13105726872246695
Empirical FPR: 0.29225736095965105
FNR alpha estimate: 0.10553629481031597
FNR alpha estimate: 0.29408667177420744


### LTT

In [43]:
N = 1000
delta = 0.01
lambdas = np.linspace(0,1,N) # Commonly choose N = 1000
FNR = np.zeros(N)
FPR = np.zeros(N)# Compute the loss function next

for i in range(N):
    FNR[i] = risk_metrics(cal_sgmd >= lambdas[i], cal_gt)['FNR']
    FPR[i] = risk_metrics(cal_sgmd >= lambdas[i], cal_gt)['FPR']
    
risk = FNR
pvals = np.exp(-2*n*(np.maximum(0.0, alpha - risk)**2)) # Or any p-value
lambda_hat = lambdas[pvals < delta/lambdas.shape[0]]
lambda_hat.max()

0.16716716716716717

In [53]:
n

5488

In [52]:
np.exp(-2*n*(np.maximum(0.0, alpha - min(risk))**2))

2.147027792703121e-48

### Inverse LTT

In [39]:
lam = 0.165

In [41]:
N = 1000
delta = 0.01
alpha_space = np.linspace(0,1,N) # Commonly choose N = 1000
pvals_FNR = np.zeros(N)
pvals_FPR = np.zeros(N)# Compute the loss function next

FNR = risk_metrics(cal_sgmd >= lam, cal_gt)['FNR']
FPR = risk_metrics(cal_sgmd >= lam, cal_gt)['FPR']
    
for i in range(N):
    pvals_FNR[i] = np.exp(-2*n*(np.maximum(0.0, alpha_space[i] - FNR)**2)) # Or any p-value
    pvals_FPR[i] = np.exp(-2*n*(np.maximum(0.0, alpha_space[i] - FPR)**2)) # Or any p-value
    
alpha_hat_FNR = alpha_space[pvals_FNR < delta/alpha_space.shape[0]]
alpha_hat_FPR = alpha_space[pvals_FPR < delta/alpha_space.shape[0]]

alpha_hat_FPR.min(), alpha_hat_FNR.min()

(0.4034034034034034, 0.0980980980980981)